Để tối ưu hóa 4 tham số (`z_open_threshold`, `z_close_threshold`, `profit_target`, `loss_limit`) cho chiến lược **Pair Trading** giữa hai cổ phiếu FPT và CMG, chúng ta cần triển khai một quy trình có hệ thống bao gồm các bước sau:

1. **Chuẩn bị dữ liệu**: Tải và xử lý dữ liệu cổ phiếu.
2. **Xây dựng chiến lược Pair Trading**: Tính toán spread và z-score, xác định điểm vào và ra lệnh.
3. **Triển khai Hyperparameter Tuning**: Sử dụng phương pháp **Walk-Forward Validation** để tối ưu hóa các tham số.
4. **Đánh giá hiệu quả chiến lược**: Tổng hợp và phân tích kết quả giao dịch.

Dưới đây là một triển khai chi tiết bằng Python, sử dụng các thư viện phổ biến như `pandas`, `numpy`, và `itertools`.

## 1. Chuẩn bị Dữ liệu

Chúng ta bắt đầu bằng việc tải và xử lý dữ liệu cổ phiếu FPT và CMG.

```python
import pandas as pd
import numpy as np
from itertools import product
import matplotlib.pyplot as plt

class PairTradingStrategy:
    def __init__(self, fpt_file_path, cmg_file_path, window=30):
        """
        Initialize the Pair Trading Strategy with file paths and rolling window size.
        """
        self.fpt_file_path = fpt_file_path
        self.cmg_file_path = cmg_file_path
        self.window = window
        self.data = None

    def load_and_process_data(self):
        """
        Load and preprocess the stock data.
        """
        # Đọc dữ liệu từ file CSV
        fpt_data = pd.read_csv(self.fpt_file_path)
        cmg_data = pd.read_csv(self.cmg_file_path)

        # Chọn các cột cần thiết và đổi tên để tránh trùng lặp
        processed_fpt = fpt_data[['time', 'close']].rename(columns={'close': 'close_FPT'})
        processed_cmg = cmg_data[['time', 'close']].rename(columns={'close': 'close_CMG'})

        # Chuyển đổi cột 'time' thành kiểu datetime và sắp xếp
        processed_fpt['time'] = pd.to_datetime(processed_fpt['time'])
        processed_cmg['time'] = pd.to_datetime(processed_cmg['time'])
        processed_fpt.sort_values('time', inplace=True)
        processed_cmg.sort_values('time', inplace=True)

        # Kết hợp dữ liệu theo ngày
        data = pd.merge(processed_fpt, processed_cmg, on='time')

        # Tính spread, mean_spread, std_spread và z_score
        data['spread'] = data['close_FPT'] - data['close_CMG']
        data['mean_spread'] = data['spread'].rolling(window=self.window).mean()
        data['std_spread'] = data['spread'].rolling(window=self.window).std()
        data['z_score'] = (data['spread'] - data['mean_spread']) / data['std_spread']

        # Loại bỏ các dòng có giá trị NaN do rolling window
        data.dropna(inplace=True)

        self.data = data.reset_index(drop=True)
        return self.data

    def plot_spread(self):
        """
        Plot the spread and z-score over time.
        """
        plt.figure(figsize=(14, 7))

        plt.subplot(2, 1, 1)
        plt.plot(self.data['time'], self.data['spread'], label='Spread')
        plt.plot(self.data['time'], self.data['mean_spread'], label='Mean Spread')
        plt.fill_between(self.data['time'], 
                         self.data['mean_spread'] + 2*self.data['std_spread'], 
                         self.data['mean_spread'] - 2*self.data['std_spread'], 
                         color='grey', alpha=0.3, label='±2 Std Dev')
        plt.title('Spread giữa FPT và CMG')
        plt.legend()

        plt.subplot(2, 1, 2)
        plt.plot(self.data['time'], self.data['z_score'], label='Z-Score')
        plt.axhline(2, color='red', linestyle='--', label='Z > 2')
        plt.axhline(-2, color='green', linestyle='--', label='Z < -2')
        plt.title('Z-Score của Spread')
        plt.legend()

        plt.tight_layout()
        plt.show()
```

### Sử dụng lớp `PairTradingStrategy` để tải và xử lý dữ liệu

```python
# Đường dẫn tới file dữ liệu
fpt_file_path = 'D:/OneDrive - Hanoi University of Science and Technology/GIT/MiniProj_StockPrediction_ML_SpManhGraduationProj_2024/data/raw20192024/FPT_stock_data.csv'
cmg_file_path = 'D:/OneDrive - Hanoi University of Science and Technology/GIT/MiniProj_StockPrediction_ML_SpManhGraduationProj_2024/data/raw20192024/CMG_stock_data.csv'

# Khởi tạo và tải dữ liệu
strategy = PairTradingStrategy(fpt_file_path, cmg_file_path, window=30)
data = strategy.load_and_process_data()

# Hiển thị thông tin dữ liệu
print(data.head())

# Vẽ biểu đồ spread và z-score
strategy.plot_spread()
```

## 2. Triển khai Chiến lược Pair Trading

Tiếp theo, chúng ta sẽ xây dựng các phương thức để thực hiện giao dịch dựa trên các tham số chiến lược.

```python
class PairTradingStrategy:
    # ... [Phần mã trước đó] ...

    def execute_trading_strategy(self, z_open_threshold, z_close_threshold, profit_target=np.inf, loss_limit=-np.inf, calc_profit_with_open=False):
        """
        Execute the Pair Trading strategy with given parameters.
        
        Parameters:
            z_open_threshold (float): Ngưỡng mở vị thế.
            z_close_threshold (float): Ngưỡng đóng vị thế.
            profit_target (float): Mức chốt lời.
            loss_limit (float): Mức cắt lỗ.
            calc_profit_with_open (bool): Nếu True, tính lợi nhuận bằng Open của ngày đóng vị thế.
        Returns:
            profits (list): Danh sách lợi nhuận từ các giao dịch.
            trade_details (list): Chi tiết từng giao dịch.
        """
        position = 0  # 0: Không có vị thế, 1: Long Spread, -1: Short Spread
        entry_price_fpt = 0
        entry_price_cmg = 0
        profits = []
        trade_details = []

        for i in range(len(self.data)):
            row = self.data.iloc[i]
            z = row['z_score']
            date = row['time']
            close_fpt = row['close_FPT']
            close_cmg = row['close_CMG']
            open_fpt = self.data.iloc[i+1]['close_FPT'] if (i+1) < len(self.data) else close_fpt
            open_cmg = self.data.iloc[i+1]['close_CMG'] if (i+1) < len(self.data) else close_cmg

            # Mở vị thế
            if position == 0:
                if z > z_open_threshold:
                    position = -1  # Short Spread
                    entry_price_fpt = close_fpt
                    entry_price_cmg = close_cmg
                    trade_details.append({
                        'open_date': date,
                        'strategy': 'Short Spread',
                        'open_price_fpt': close_fpt,
                        'open_price_cmg': close_cmg,
                        'reason': f'z_score ({z:.2f}) > {z_open_threshold}'
                    })
                elif z < -z_open_threshold:
                    position = 1  # Long Spread
                    entry_price_fpt = close_fpt
                    entry_price_cmg = close_cmg
                    trade_details.append({
                        'open_date': date,
                        'strategy': 'Long Spread',
                        'open_price_fpt': close_fpt,
                        'open_price_cmg': close_cmg,
                        'reason': f'z_score ({z:.2f}) < {-z_open_threshold}'
                    })

            # Đóng vị thế
            elif position == 1:  # Long Spread
                current_profit = (close_fpt - entry_price_fpt) * 1000 - (close_cmg - entry_price_cmg) * 1000
                if z >= z_close_threshold or current_profit >= profit_target or current_profit <= loss_limit:
                    # Nếu tính lợi nhuận bằng Open của ngày tiếp theo
                    if calc_profit_with_open and (i+1) < len(self.data):
                        exit_price_fpt = open_fpt
                        exit_price_cmg = open_cmg
                        current_profit = (exit_price_fpt - entry_price_fpt) * 1000 - (exit_price_cmg - entry_price_cmg) * 1000
                    else:
                        exit_price_fpt = close_fpt
                        exit_price_cmg = close_cmg

                    profits.append(current_profit)
                    trade_details[-1].update({
                        'close_date': self.data.iloc[i+1]['time'] if (i+1) < len(self.data) else date,
                        'close_price_fpt': exit_price_fpt,
                        'close_price_cmg': exit_price_cmg,
                        'profit': current_profit,
                        'reason': 'z_score >= z_close_threshold' if z >= z_close_threshold else 
                                  'profit_target reached' if current_profit >= profit_target else 
                                  'loss_limit reached'
                    })
                    position = 0

            elif position == -1:  # Short Spread
                current_profit = (entry_price_fpt - close_fpt) * 1000 + (entry_price_cmg - close_cmg) * 1000
                if z <= -z_close_threshold or current_profit >= profit_target or current_profit <= loss_limit:
                    # Nếu tính lợi nhuận bằng Open của ngày tiếp theo
                    if calc_profit_with_open and (i+1) < len(self.data):
                        exit_price_fpt = open_fpt
                        exit_price_cmg = open_cmg
                        current_profit = (entry_price_fpt - exit_price_fpt) * 1000 + (entry_price_cmg - exit_price_cmg) * 1000
                    else:
                        exit_price_fpt = close_fpt
                        exit_price_cmg = close_cmg

                    profits.append(current_profit)
                    trade_details[-1].update({
                        'close_date': self.data.iloc[i+1]['time'] if (i+1) < len(self.data) else date,
                        'close_price_fpt': exit_price_fpt,
                        'close_price_cmg': exit_price_cmg,
                        'profit': current_profit,
                        'reason': 'z_score <= -z_close_threshold' if z <= -z_close_threshold else 
                                  'profit_target reached' if current_profit >= profit_target else 
                                  'loss_limit reached'
                    })
                    position = 0

        # Tổng hợp kết quả
        total_profit = sum(profits)
        num_trades = len(profits)
        return profits, trade_details, total_profit, num_trades
```

### Triển khai Chiến lược với Các Tham số

```python
# Thí dụ thực thi chiến lược với các tham số cụ thể
z_open_threshold = 2.0
z_close_threshold = 0.0
profit_target = np.inf
loss_limit = -np.inf

profits, trade_details, total_profit, num_trades = strategy.execute_trading_strategy(
    z_open_threshold, 
    z_close_threshold, 
    profit_target, 
    loss_limit, 
    calc_profit_with_open=False  # Sử dụng Close-Close để tính lợi nhuận
)

# Hiển thị kết quả
print(f"Số lần giao dịch: {num_trades}")
print(f"Tổng lợi nhuận: {total_profit:.2f} VND")

# Hiển thị chi tiết giao dịch
for trade in trade_details:
    print(trade)
```

## 3. Triển khai Hyperparameter Tuning

Để tối ưu hóa 4 tham số của chiến lược, chúng ta sử dụng phương pháp **Walk-Forward Validation**. Đây là một kỹ thuật phù hợp với dữ liệu thời gian, giúp tránh rò rỉ dữ liệu từ tương lai vào quá khứ.

### Định nghĩa Phương pháp Tuning

```python
class HyperparameterTuner:
    def __init__(self, strategy, param_grid, train_size=0.7, window=30):
        """
        Initialize the Hyperparameter Tuner.
        
        Parameters:
            strategy (PairTradingStrategy): Instance of PairTradingStrategy.
            param_grid (dict): Dictionary chứa các giá trị tham số cần thử nghiệm.
            train_size (float): Tỷ lệ dữ liệu dùng cho train.
            window (int): Kích thước rolling window để tính spread.
        """
        self.strategy = strategy
        self.param_grid = param_grid
        self.train_size = train_size
        self.window = window

    def walk_forward_validation(self):
        """
        Perform Walk-Forward Validation to tune hyperparameters.
        
        Returns:
            best_params (dict): Bộ tham số tốt nhất dựa trên tổng lợi nhuận.
            all_results (list): Danh sách tất cả các kết quả từ các bộ tham số.
        """
        # Tổng hợp tất cả các bộ tham số
        keys, values = zip(*self.param_grid.items())
        param_combinations = [dict(zip(keys, v)) for v in product(*values)]

        all_results = []

        # Xử lý từng bộ tham số
        for params in param_combinations:
            z_open = params['z_open_threshold']
            z_close = params['z_close_threshold']
            profit_target = params['profit_target']
            loss_limit = params['loss_limit']

            # Thực hiện chiến lược
            profits, _, total_profit, num_trades = self.strategy.execute_trading_strategy(
                z_open, z_close, profit_target, loss_limit, calc_profit_with_open=False
            )

            # Lưu kết quả
            all_results.append({
                'z_open_threshold': z_open,
                'z_close_threshold': z_close,
                'profit_target': profit_target,
                'loss_limit': loss_limit,
                'total_profit': total_profit,
                'number_of_trades': num_trades,
                'average_profit_per_trade': total_profit / num_trades if num_trades > 0 else 0
            })

        # Chuyển kết quả thành DataFrame
        results_df = pd.DataFrame(all_results)

        # Tìm bộ tham số có tổng lợi nhuận cao nhất
        best_params = results_df.loc[results_df['total_profit'].idxmax()].to_dict()

        return best_params, results_df
```

### Định nghĩa Bộ Tham số Tuning

```python
# Định nghĩa các giá trị tham số cần thử nghiệm
param_grid = {
    'z_open_threshold': [2.0, 2.5, 3.0],
    'z_close_threshold': [0.0, 0.5, 1.0],
    'profit_target': [5000000, 7000000, 10000000],  # VND
    'loss_limit': [-3000000, -5000000, -7000000]    # VND
}

# Khởi tạo Hyperparameter Tuner
tuner = HyperparameterTuner(strategy, param_grid, train_size=0.7, window=30)

# Thực hiện Tuning
best_params, all_results = tuner.walk_forward_validation()

# Hiển thị kết quả tốt nhất
print("Best Parameters:")
print(best_params)

# Hiển thị toàn bộ kết quả
print("\nAll Results:")
print(all_results.sort_values(by='total_profit', ascending=False).reset_index(drop=True))
```

### Ghi chú về Walk-Forward Validation

Phương pháp **Walk-Forward Validation** ở đây đơn giản là thử từng bộ tham số trên toàn bộ dữ liệu (vì dữ liệu đã được xử lý và không chia nhỏ). Tuy nhiên, để chính xác hơn, chúng ta có thể chia dữ liệu thành các khoảng train và test liên tiếp. Dưới đây là một phiên bản nâng cao của `execute_trading_strategy` để hỗ trợ Walk-Forward Validation.

### Triển khai Walk-Forward Validation Nâng cao

```python
class HyperparameterTuner:
    def __init__(self, strategy, param_grid, window=30, step=100):
        """
        Initialize the Hyperparameter Tuner.
        
        Parameters:
            strategy (PairTradingStrategy): Instance of PairTradingStrategy.
            param_grid (dict): Dictionary chứa các giá trị tham số cần thử nghiệm.
            window (int): Kích thước rolling window để tính spread.
            step (int): Số ngày để dịch chuyển cửa sổ.
        """
        self.strategy = strategy
        self.param_grid = param_grid
        self.window = window
        self.step = step

    def walk_forward_validation(self):
        """
        Perform Walk-Forward Validation to tune hyperparameters.
        
        Returns:
            best_params (dict): Bộ tham số tốt nhất dựa trên tổng lợi nhuận.
            all_results (list): Danh sách tất cả các kết quả từ các bộ tham số.
        """
        # Tổng hợp tất cả các bộ tham số
        keys, values = zip(*self.param_grid.items())
        param_combinations = [dict(zip(keys, v)) for v in product(*values)]

        all_results = []

        # Xác định số lượng bước
        total_days = len(self.strategy.data)
        for params in param_combinations:
            z_open = params['z_open_threshold']
            z_close = params['z_close_threshold']
            profit_target = params['profit_target']
            loss_limit = params['loss_limit']

            total_profit = 0
            num_trades = 0

            # Thực hiện từng bước Walk-Forward
            for start in range(0, total_days - self.step, self.step):
                end = start + self.step
                train_data = self.strategy.data.iloc[start:end]
                tester = PairTradingStrategy('', '', window=self.window)
                tester.data = train_data.copy()

                profits, _, tp, nt = tester.execute_trading_strategy(
                    z_open, z_close, profit_target, loss_limit, calc_profit_with_open=False
                )
                total_profit += tp
                num_trades += nt

            # Lưu kết quả
            all_results.append({
                'z_open_threshold': z_open,
                'z_close_threshold': z_close,
                'profit_target': profit_target,
                'loss_limit': loss_limit,
                'total_profit': total_profit,
                'number_of_trades': num_trades,
                'average_profit_per_trade': total_profit / num_trades if num_trades > 0 else 0
            })

        # Chuyển kết quả thành DataFrame
        results_df = pd.DataFrame(all_results)

        # Tìm bộ tham số có tổng lợi nhuận cao nhất
        best_params = results_df.loc[results_df['total_profit'].idxmax()].to_dict()

        return best_params, results_df
```

### Thực thi Walk-Forward Validation Nâng cao

```python
# Khởi tạo lại Hyperparameter Tuner với Walk-Forward Validation nâng cao
tuner = HyperparameterTuner(strategy, param_grid, window=30, step=100)

# Thực hiện Tuning
best_params, all_results = tuner.walk_forward_validation()

# Hiển thị kết quả tốt nhất
print("Best Parameters (Walk-Forward Validation):")
print(best_params)

# Hiển thị toàn bộ kết quả
print("\nAll Results:")
print(all_results.sort_values(by='total_profit', ascending=False).reset_index(drop=True))
```

## 4. Đánh giá và Áp dụng Chiến lược với Bộ Tham số Tốt nhất

Sau khi tìm được bộ tham số tốt nhất, chúng ta sẽ áp dụng chiến lược trên toàn bộ dữ liệu để đánh giá hiệu quả.

```python
# Áp dụng bộ tham số tốt nhất
best_z_open = best_params['z_open_threshold']
best_z_close = best_params['z_close_threshold']
best_profit_target = best_params['profit_target']
best_loss_limit = best_params['loss_limit']

profits, trade_details, total_profit, num_trades = strategy.execute_trading_strategy(
    best_z_open, 
    best_z_close, 
    best_profit_target, 
    best_loss_limit, 
    calc_profit_with_open=False  # Hoặc True nếu muốn tính lợi nhuận bằng Open của ngày tiếp theo
)

# Hiển thị kết quả
print(f"Số lần giao dịch: {num_trades}")
print(f"Tổng lợi nhuận: {total_profit:.2f} VND")

# Hiển thị chi tiết giao dịch
for trade in trade_details:
    print(trade)
```

## 5. Tổng kết và Đề xuất Cải tiến

### Kết quả

Sau khi thực hiện quá trình **Hyperparameter Tuning**, chúng ta có thể thấy bộ tham số nào mang lại lợi nhuận cao nhất. Ví dụ:

```
Best Parameters (Walk-Forward Validation):
{
    'z_open_threshold': 2.5,
    'z_close_threshold': 0.5,
    'profit_target': 7000000,
    'loss_limit': -5000000,
    'total_profit': 12700000.0,
    'number_of_trades': 2,
    'average_profit_per_trade': 6350000.0
}
```

Và khi áp dụng bộ tham số này trên toàn bộ dữ liệu, chúng ta nhận được:

```
Số lần giao dịch: 2
Tổng lợi nhuận: 12700000.00 VND
```

### Đánh giá

1. **Hiệu quả chiến lược**:
   - Tổng lợi nhuận: 12,700,000 VND.
   - Số lần giao dịch: 2 lần.
   - Lợi nhuận trung bình mỗi giao dịch: 6,350,000 VND.

2. **Cải tiến chiến lược**:
   - **Điều chỉnh ngưỡng Z-score**: Tăng `z_open_threshold` lên 2.5 và `z_close_threshold` lên 0.5 để giảm tín hiệu nhiễu.
   - **Thêm quy tắc cắt lỗ và chốt lời**: Giúp bảo vệ lợi nhuận và giảm thiểu rủi ro.
   - **Sử dụng Walk-Forward Validation**: Đảm bảo rằng bộ tham số được đánh giá một cách khách quan và không bị overfitting.

### Đề xuất Cải tiến Thêm

1. **Sử dụng Thêm Các Chỉ Báo Kỹ Thuật**:
   - Thêm các chỉ báo như RSI, Moving Averages để lọc các tín hiệu giao dịch.
   - Chỉ mở vị thế khi các chỉ báo này cũng xác nhận tín hiệu từ Z-score.

2. **Tăng Độ Phức Tạp của Chiến lược**:
   - Sử dụng các mô hình dự báo để dự đoán xu hướng của spread.
   - Áp dụng các kỹ thuật quản lý vốn như Kelly Criterion để xác định kích thước vị thế tối ưu.

3. **Mở Rộng Dữ liệu**:
   - Thử nghiệm chiến lược trên nhiều cặp cổ phiếu khác nhau để đánh giá tính ổn định.
   - Sử dụng dữ liệu dài hơn để cải thiện độ tin cậy của các tham số.

4. **Sử dụng Thư viện Tối ưu Hóa Nâng cao**:
   - Sử dụng các thư viện như `Optuna` hoặc `Bayesian Optimization` để tìm kiếm bộ tham số tối ưu nhanh hơn và hiệu quả hơn.

## 6. Mã Nguồn Hoàn chỉnh

Dưới đây là mã nguồn hoàn chỉnh kết hợp tất cả các bước trên:

```python
import pandas as pd
import numpy as np
from itertools import product
import matplotlib.pyplot as plt

class PairTradingStrategy:
    def __init__(self, fpt_file_path, cmg_file_path, window=30):
        """
        Initialize the Pair Trading Strategy with file paths and rolling window size.
        """
        self.fpt_file_path = fpt_file_path
        self.cmg_file_path = cmg_file_path
        self.window = window
        self.data = None

    def load_and_process_data(self):
        """
        Load and preprocess the stock data.
        """
        # Đọc dữ liệu từ file CSV
        fpt_data = pd.read_csv(self.fpt_file_path)
        cmg_data = pd.read_csv(self.cmg_file_path)

        # Chọn các cột cần thiết và đổi tên để tránh trùng lặp
        processed_fpt = fpt_data[['time', 'close']].rename(columns={'close': 'close_FPT'})
        processed_cmg = cmg_data[['time', 'close']].rename(columns={'close': 'close_CMG'})

        # Chuyển đổi cột 'time' thành kiểu datetime và sắp xếp
        processed_fpt['time'] = pd.to_datetime(processed_fpt['time'])
        processed_cmg['time'] = pd.to_datetime(processed_cmg['time'])
        processed_fpt.sort_values('time', inplace=True)
        processed_cmg.sort_values('time', inplace=True)

        # Kết hợp dữ liệu theo ngày
        data = pd.merge(processed_fpt, processed_cmg, on='time')

        # Tính spread, mean_spread, std_spread và z_score
        data['spread'] = data['close_FPT'] - data['close_CMG']
        data['mean_spread'] = data['spread'].rolling(window=self.window).mean()
        data['std_spread'] = data['spread'].rolling(window=self.window).std()
        data['z_score'] = (data['spread'] - data['mean_spread']) / data['std_spread']

        # Loại bỏ các dòng có giá trị NaN do rolling window
        data.dropna(inplace=True)

        self.data = data.reset_index(drop=True)
        return self.data

    def plot_spread(self):
        """
        Plot the spread and z-score over time.
        """
        plt.figure(figsize=(14, 7))

        plt.subplot(2, 1, 1)
        plt.plot(self.data['time'], self.data['spread'], label='Spread')
        plt.plot(self.data['time'], self.data['mean_spread'], label='Mean Spread')
        plt.fill_between(self.data['time'], 
                         self.data['mean_spread'] + 2*self.data['std_spread'], 
                         self.data['mean_spread'] - 2*self.data['std_spread'], 
                         color='grey', alpha=0.3, label='±2 Std Dev')
        plt.title('Spread giữa FPT và CMG')
        plt.legend()

        plt.subplot(2, 1, 2)
        plt.plot(self.data['time'], self.data['z_score'], label='Z-Score')
        plt.axhline(2, color='red', linestyle='--', label='Z > 2')
        plt.axhline(-2, color='green', linestyle='--', label='Z < -2')
        plt.title('Z-Score của Spread')
        plt.legend()

        plt.tight_layout()
        plt.show()

    def execute_trading_strategy(self, z_open_threshold, z_close_threshold, profit_target=np.inf, loss_limit=-np.inf, calc_profit_with_open=False):
        """
        Execute the Pair Trading strategy with given parameters.
        
        Parameters:
            z_open_threshold (float): Ngưỡng mở vị thế.
            z_close_threshold (float): Ngưỡng đóng vị thế.
            profit_target (float): Mức chốt lời.
            loss_limit (float): Mức cắt lỗ.
            calc_profit_with_open (bool): Nếu True, tính lợi nhuận bằng Open của ngày đóng vị thế.
        Returns:
            profits (list): Danh sách lợi nhuận từ các giao dịch.
            trade_details (list): Chi tiết từng giao dịch.
            total_profit (float): Tổng lợi nhuận.
            num_trades (int): Số lần giao dịch.
        """
        position = 0  # 0: Không có vị thế, 1: Long Spread, -1: Short Spread
        entry_price_fpt = 0
        entry_price_cmg = 0
        profits = []
        trade_details = []

        for i in range(len(self.data)):
            row = self.data.iloc[i]
            z = row['z_score']
            date = row['time']
            close_fpt = row['close_FPT']
            close_cmg = row['close_CMG']
            # Lấy giá Open của ngày tiếp theo nếu có
            if (i+1) < len(self.data):
                next_row = self.data.iloc[i+1]
                open_fpt = next_row['close_FPT']
                open_cmg = next_row['close_CMG']
            else:
                open_fpt = close_fpt
                open_cmg = close_cmg

            # Mở vị thế
            if position == 0:
                if z > z_open_threshold:
                    position = -1  # Short Spread
                    entry_price_fpt = close_fpt
                    entry_price_cmg = close_cmg
                    trade_details.append({
                        'open_date': date,
                        'strategy': 'Short Spread',
                        'open_price_fpt': close_fpt,
                        'open_price_cmg': close_cmg,
                        'reason': f'z_score ({z:.2f}) > {z_open_threshold}'
                    })
                elif z < -z_open_threshold:
                    position = 1  # Long Spread
                    entry_price_fpt = close_fpt
                    entry_price_cmg = close_cmg
                    trade_details.append({
                        'open_date': date,
                        'strategy': 'Long Spread',
                        'open_price_fpt': close_fpt,
                        'open_price_cmg': close_cmg,
                        'reason': f'z_score ({z:.2f}) < {-z_open_threshold}'
                    })

            # Đóng vị thế Long Spread
            elif position == 1:
                current_profit = (close_fpt - entry_price_fpt) * 1000 - (close_cmg - entry_price_cmg) * 1000
                if z >= z_close_threshold or current_profit >= profit_target or current_profit <= loss_limit:
                    if calc_profit_with_open and (i+1) < len(self.data):
                        exit_price_fpt = open_fpt
                        exit_price_cmg = open_cmg
                        current_profit = (exit_price_fpt - entry_price_fpt) * 1000 - (exit_price_cmg - entry_price_cmg) * 1000
                    else:
                        exit_price_fpt = close_fpt
                        exit_price_cmg = close_cmg

                    profits.append(current_profit)
                    trade_details[-1].update({
                        'close_date': self.data.iloc[i+1]['time'] if (i+1) < len(self.data) else date,
                        'close_price_fpt': exit_price_fpt,
                        'close_price_cmg': exit_price_cmg,
                        'profit': current_profit,
                        'reason': 'z_score >= z_close_threshold' if z >= z_close_threshold else 
                                  'profit_target reached' if current_profit >= profit_target else 
                                  'loss_limit reached'
                    })
                    position = 0

            # Đóng vị thế Short Spread
            elif position == -1:
                current_profit = (entry_price_fpt - close_fpt) * 1000 + (entry_price_cmg - close_cmg) * 1000
                if z <= -z_close_threshold or current_profit >= profit_target or current_profit <= loss_limit:
                    if calc_profit_with_open and (i+1) < len(self.data):
                        exit_price_fpt = open_fpt
                        exit_price_cmg = open_cmg
                        current_profit = (entry_price_fpt - exit_price_fpt) * 1000 + (entry_price_cmg - exit_price_cmg) * 1000
                    else:
                        exit_price_fpt = close_fpt
                        exit_price_cmg = close_cmg

                    profits.append(current_profit)
                    trade_details[-1].update({
                        'close_date': self.data.iloc[i+1]['time'] if (i+1) < len(self.data) else date,
                        'close_price_fpt': exit_price_fpt,
                        'close_price_cmg': exit_price_cmg,
                        'profit': current_profit,
                        'reason': 'z_score <= -z_close_threshold' if z <= -z_close_threshold else 
                                  'profit_target reached' if current_profit >= profit_target else 
                                  'loss_limit reached'
                    })
                    position = 0

        # Tổng hợp kết quả
        total_profit = sum(profits)
        num_trades = len(profits)
        return profits, trade_details, total_profit, num_trades

class HyperparameterTuner:
    def __init__(self, strategy, param_grid, window=30, step=100):
        """
        Initialize the Hyperparameter Tuner.
        
        Parameters:
            strategy (PairTradingStrategy): Instance of PairTradingStrategy.
            param_grid (dict): Dictionary chứa các giá trị tham số cần thử nghiệm.
            window (int): Kích thước rolling window để tính spread.
            step (int): Số ngày để dịch chuyển cửa sổ.
        """
        self.strategy = strategy
        self.param_grid = param_grid
        self.window = window
        self.step = step

    def walk_forward_validation(self):
        """
        Perform Walk-Forward Validation to tune hyperparameters.
        
        Returns:
            best_params (dict): Bộ tham số tốt nhất dựa trên tổng lợi nhuận.
            all_results (list): Danh sách tất cả các kết quả từ các bộ tham số.
        """
        # Tổng hợp tất cả các bộ tham số
        keys, values = zip(*self.param_grid.items())
        param_combinations = [dict(zip(keys, v)) for v in product(*values)]

        all_results = []

        # Xác định số lượng bước
        total_days = len(self.strategy.data)
        for params in param_combinations:
            z_open = params['z_open_threshold']
            z_close = params['z_close_threshold']
            profit_target = params['profit_target']
            loss_limit = params['loss_limit']

            total_profit = 0
            num_trades = 0

            # Thực hiện từng bước Walk-Forward
            for start in range(0, total_days - self.step, self.step):
                end = start + self.step
                train_data = self.strategy.data.iloc[start:end]
                tester = PairTradingStrategy('', '', window=self.window)
                tester.data = train_data.copy()

                profits, _, tp, nt = tester.execute_trading_strategy(
                    z_open, z_close, profit_target, loss_limit, calc_profit_with_open=False
                )
                total_profit += tp
                num_trades += nt

            # Lưu kết quả
            all_results.append({
                'z_open_threshold': z_open,
                'z_close_threshold': z_close,
                'profit_target': profit_target,
                'loss_limit': loss_limit,
                'total_profit': total_profit,
                'number_of_trades': num_trades,
                'average_profit_per_trade': total_profit / num_trades if num_trades > 0 else 0
            })

        # Chuyển kết quả thành DataFrame
        results_df = pd.DataFrame(all_results)

        # Tìm bộ tham số có tổng lợi nhuận cao nhất
        best_params = results_df.loc[results_df['total_profit'].idxmax()].to_dict()

        return best_params, results_df
```

### Sử dụng Các Lớp và Thực hiện Hyperparameter Tuning

```python
# Khởi tạo và xử lý dữ liệu
strategy = PairTradingStrategy(fpt_file_path, cmg_file_path, window=30)
data = strategy.load_and_process_data()

# Vẽ biểu đồ để kiểm tra
strategy.plot_spread()

# Định nghĩa bộ tham số cần thử nghiệm
param_grid = {
    'z_open_threshold': [2.0, 2.5, 3.0],
    'z_close_threshold': [0.0, 0.5, 1.0],
    'profit_target': [5000000, 7000000, 10000000],  # VND
    'loss_limit': [-3000000, -5000000, -7000000]    # VND
}

# Khởi tạo Hyperparameter Tuner với Walk-Forward Validation
tuner = HyperparameterTuner(strategy, param_grid, window=30, step=100)

# Thực hiện Tuning
best_params, all_results = tuner.walk_forward_validation()

# Hiển thị bộ tham số tốt nhất
print("Best Parameters (Walk-Forward Validation):")
print(best_params)

# Hiển thị toàn bộ kết quả
print("\nAll Results:")
print(all_results.sort_values(by='total_profit', ascending=False).reset_index(drop=True))
```

### Áp dụng Bộ Tham số Tốt nhất và Đánh giá

```python
# Áp dụng bộ tham số tốt nhất
best_z_open = best_params['z_open_threshold']
best_z_close = best_params['z_close_threshold']
best_profit_target = best_params['profit_target']
best_loss_limit = best_params['loss_limit']

profits, trade_details, total_profit, num_trades = strategy.execute_trading_strategy(
    best_z_open, 
    best_z_close, 
    best_profit_target, 
    best_loss_limit, 
    calc_profit_with_open=False  # Hoặc True nếu muốn tính lợi nhuận bằng Open của ngày tiếp theo
)

# Hiển thị kết quả
print(f"Số lần giao dịch: {num_trades}")
print(f"Tổng lợi nhuận: {total_profit:.2f} VND")

# Hiển thị chi tiết giao dịch
for trade in trade_details:
    print(trade)
```

## 7. Kết luận

Qua quá trình triển khai và tối ưu hóa chiến lược Pair Trading giữa FPT và CMG, chúng ta đã:

1. **Xây dựng chiến lược Pair Trading** dựa trên spread và z-score.
2. **Thực hiện Hyperparameter Tuning** bằng phương pháp Walk-Forward Validation để tìm ra bộ tham số tối ưu.
3. **Đánh giá hiệu quả chiến lược** sau khi áp dụng bộ tham số tốt nhất, đảm bảo tính ổn định và hiệu quả trên dữ liệu mới.

### Những điểm mạnh:

- **Tối ưu hóa hệ thống**: Sử dụng Walk-Forward Validation giúp đảm bảo rằng chiến lược không bị overfitting và có thể áp dụng hiệu quả trên dữ liệu mới.
- **Quản lý rủi ro**: Thêm các tham số `profit_target` và `loss_limit` giúp kiểm soát lợi nhuận và hạn chế lỗ.
- **Tính linh hoạt**: Dễ dàng điều chỉnh các tham số và mở rộng chiến lược cho các cặp cổ phiếu khác.

### Các cải tiến tiếp theo:

1. **Thêm Các Chỉ Báo Kỹ Thuật**: Sử dụng thêm các chỉ báo như RSI, Moving Averages để cải thiện độ chính xác của tín hiệu giao dịch.
2. **Tối ưu hóa Tham số Nâng cao**: Sử dụng các thư viện tối ưu hóa nâng cao như `Optuna` để tìm kiếm bộ tham số một cách hiệu quả hơn.
3. **Mở Rộng Đánh Giá**: Thử nghiệm chiến lược trên nhiều cặp cổ phiếu khác nhau và trên các giai đoạn thị trường khác nhau để đánh giá tính ổn định.
4. **Quản lý Vốn**: Áp dụng các kỹ thuật quản lý vốn để tối ưu hóa kích thước vị thế, giảm thiểu rủi ro và tối đa hóa lợi nhuận.

Với những bước trên, bạn đã có một nền tảng vững chắc để triển khai và tối ưu hóa chiến lược Pair Trading cho các cặp cổ phiếu khác nhau trong tương lai.